In [1]:
import japanize_matplotlib  # noqa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from lightning.pytorch import seed_everything
from scipy import stats
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_array, check_X_y

plt.style.use("ggplot")
seed_everything(8)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\HaruMomozu\Desktop\momozu\ABtesting\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\HaruMomozu\Desktop\momozu\ABtesting\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\HaruMomozu\Desktop\momozu\ABtesting\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in

8

In [50]:
# 外れ値の除去
def remove_outliers_zscore(data, metric, threshold=2):
    z_scores = np.abs(stats.zscore(data[metric]))
    data = data[(z_scores < threshold)]
    return data


df1 = pd.read_csv(
    R"C:\Users\HaruMomozu\Documents\オンラインデータ\NHANES_age_prediction.csv"
)
df1 = df1.drop(columns=["SEQN", "age_group"])

obj1 = "BMXBMI"
features_list1 = [
    "RIDAGEYR",  # 年齢（連続変数）
    "RIAGENDR",  # 性別（1:Male, 2:Female)
    "PAQ605",  # 運動有無(1:日常的に運動する, 2:運動しない)
    "LBXGLU",  # 断食後の血糖値（連続変数）
    "DIQ010",  # 糖尿病の有無(0:なし、1:あり)
    "LBXGLT",  # 口内の健康状態（連続変数）
    "LBXIN",  # 血中インスリン濃度（連続変数）
]
df1 = df1[df1["PAQ605"] != 7.0]

df = df1  # 選ぶ
obj = obj1  # 選ぶ
features_list = features_list1  # 選ぶ

df = remove_outliers_zscore(df, obj)

X = df[features_list]
# 数値列の標準化
scaler = StandardScaler()
scaled_features = scaler.fit_transform(X)
X_scaled = pd.DataFrame(scaled_features, columns=features_list)

y = df[obj]  # 目的変数

# 行を詰める
df = df.reset_index(drop=True)

In [62]:
from scipy.stats import multivariate_normal


class Wrapper(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        n_features_to_select,
        n_clusters,
        criterion="ml",
        clustering_method="em",
        random_state=None,
    ):
        self.n_features_to_select = n_features_to_select  # 特徴量数
        self.n_clusters = n_clusters  # クラスタ数
        self.criterion = criterion  # 特徴量選択基準
        self.clustering_method = clustering_method  # クラスタリング手法
        self.random_state = random_state

    def FSS(self, X, y):
        X, y = check_X_y(X, y)
        self.selected_features_ = []  # 選ばれた特徴量 #最初は特徴量１のみ
        n_features = X.shape[1]  # 総特徴量数

        current_features = []
        temp_X = []
        iter = 0

        while len(current_features) < self.n_features_to_select:
            iter += 1
            best_feature = None
            if iter == 1:
                remaining_features = [i + 1 for i in range(n_features - 1)]

            for feature in remaining_features:
                temp_features = current_features + [feature]
                if iter == 1 and temp_X == []:
                    current_X = X[:, [0]]
                else:
                    current_X = temp_X
                temp_X = X[:, temp_features]
                normalized_score_current = self.calculate_normalized_score_current(
                    current_X, temp_X, y
                )
                normalized_score_temp = self.calculate_normalized_score_temp(
                    current_X, temp_X, y
                )

                if (
                    iter == 1
                    and temp_X == []
                    and normalized_score_temp <= normalized_score_current
                ):
                    best_feature = 0
                if normalized_score_temp > normalized_score_current:
                    best_feature = feature

            if iter == 1 and best_feature is not None:
                current_features.append(best_feature)
                remaining_features = [i for i in range(n_features)]
                remaining_features.remove(best_feature)
                self.selected_features_ = current_features
            elif best_feature is not None:
                current_features.append(best_feature)
                remaining_features.remove(best_feature)
                self.selected_features_ = current_features
            else:
                break

        final_features = X[:, self.selected_features_]
        if self.clustering_method == "em":
            self.final_model_ = GaussianMixture(
                n_components=self.n_clusters, random_state=self.random_state
            )
        elif self.clustering_method == "kmeans":
            self.final_model_ = KMeans(
                n_clusters=self.n_clusters, random_state=self.random_state
            )
        else:
            raise ValueError(f"Unknown clustering method: {self.clustering_method}")

        self.final_model_.fit(final_features)
        self.final_cluster_assignments_ = self.final_model_.predict(final_features)
        return self

    def calculate_normalized_score_current(self, current_X, temp_X, y):
        X1 = current_X
        X2 = temp_X
        score_current = self.CRIT(X1, X1)
        score_temp = self.CRIT(X2, X1)
        score = score_current * score_temp
        return score

    def calculate_normalized_score_temp(self, current_X, temp_X, y):
        X1 = current_X
        X2 = temp_X
        score_current = self.CRIT(X1, X2)
        score_temp = self.CRIT(X2, X2)
        score = score_current * score_temp
        return score

    def CRIT(self, X1, X2):  # X2空間上でしたクラスタリングの結果をX1空間上で評価
        if self.clustering_method == "em":
            em = GaussianMixture(
                n_components=self.n_clusters,
                random_state=self.random_state,
                init_params="kmeans",
            )
            em.fit(X2)
            labels = em.predict(X2)
        if self.clustering_method == "kmeans":
            kmeans = KMeans(
                n_clusters=self.n_clusters,
                random_state=self.random_state,
            )
            kmeans.fit(X2)
            labels = kmeans.predict(X2)

        overall_cov_mean = np.mean(np.var(X1, axis=0))
        identity_matrix = np.eye(X1.shape[1])
        avoid_singular = overall_cov_mean * identity_matrix
        cluster_num = len(np.unique(labels))

        if self.criterion == "ml":
            log_likelihood = 0.0
        if self.criterion == "tr":
            # 全体の平均ベクトル
            overall_mean = np.mean(X1, axis=0)
            # クラスター内分散行列（S_W）の計算
            S_W = np.zeros((X1.shape[1], X1.shape[1]))
            S_B = np.zeros((X1.shape[1], X1.shape[1]))

        for cluster in range(cluster_num):
            # クラスタに属するデータポイントを抽出
            cluster_points = X1[labels == cluster]
            # クラスタの平均ベクトル（μ）を計算
            mean_vector = np.mean(cluster_points, axis=0)
            # クラスタの共分散行列（Σ）を計算
            cov_matrix = np.cov(cluster_points, rowvar=False)
            if X1.shape[1] == 1:
                cov_matrix = np.array([[cov_matrix]])
            cov_matrix += avoid_singular

            if self.criterion == "ml":
                log_likelihood += np.sum(
                    multivariate_normal.logpdf(
                        cluster_points, mean=mean_vector, cov=cov_matrix
                    )
                )

            if self.criterion == "tr":
                S_W += cov_matrix * (cluster_points.shape[0] - 1)
                n_k = cluster_points.shape[0]
                mean_diff = (mean_vector - overall_mean).reshape(-1, 1)
                S_B += n_k * np.dot(mean_diff, mean_diff.T)

            if self.criterion == "ml":
                score = log_likelihood
            if self.criterion == "tr":
                score = np.trace(np.linalg.inv(S_W).dot(S_B))

        return score

    def transform(self, X):
        check_array(X)
        return X[:, self.selected_features_]

    def get_feature_index_out(self):
        return np.array(self.selected_features_)

    def get_final_cluster_assignments(self):
        return self.final_cluster_assignments_

In [65]:
import warnings

from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

clusters = 3
n_features_to_select = 3  # 選択したい特徴量の数

fssem_tr = Wrapper(
    n_features_to_select=n_features_to_select,
    n_clusters=clusters,
    criterion="tr",
    clustering_method="em",
    random_state=0,
)
fssem_ml = Wrapper(
    n_features_to_select=n_features_to_select,
    n_clusters=clusters,
    criterion="ml",
    clustering_method="em",
    random_state=0,
)
fsskmeans_tr = Wrapper(
    n_features_to_select=n_features_to_select,
    n_clusters=clusters,
    criterion="tr",
    clustering_method="kmeans",
    random_state=0,
)
fsskmeans_ml = Wrapper(
    n_features_to_select=n_features_to_select,
    n_clusters=clusters,
    criterion="ml",
    clustering_method="kmeans",
    random_state=0,
)

instance_dict = {
    "fssem_tr": fssem_tr,
    "fssem_ml": fssem_ml,
    "fsskmeans_tr": fsskmeans_tr,
    "fsskmeans_ml": fsskmeans_ml,
}

In [67]:
selected_features_index_dict = {}
cluster_label_dict = {}
cluster_size_dict = {}
for name, instance in instance_dict.items():
    instance.FSS(X_scaled, y)
    selected_features_index = instance.get_feature_index_out()
    selected_features_index_dict[name] = selected_features_index
    cluster_label = instance.get_final_cluster_assignments()
    cluster_label_dict[name] = cluster_label
    cluster_size = np.unique(cluster_label, return_counts=True)[1]
    cluster_size_dict[name] = cluster_size
print(selected_features_index_dict)
print(cluster_size_dict)

C:\Users\HaruMomozu\AppData\Local\Temp\ipykernel_9972\445121207.py:50: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  and temp_X == []
C:\Users\HaruMomozu\AppData\Local\Temp\ipykernel_9972\445121207.py:36: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if iter == 1 and temp_X == []:
C:\Users\HaruMomozu\AppData\Local\Temp\ipykernel_9972\445121207.py:50: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  and temp_X == []
C:\Users\HaruMomozu\AppData\Local\Temp\ipykernel_9972\445121207.py:36: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False,

{'fssem_tr': array([6, 5, 2]), 'fssem_ml': array([5, 4, 6]), 'fsskmeans_tr': array([6, 4, 3]), 'fsskmeans_ml': array([5, 6, 3])}
{'fssem_tr': array([ 389, 1504,  276]), 'fssem_ml': array([2072,   45,   52]), 'fsskmeans_tr': array([1792,  325,   52]), 'fsskmeans_ml': array([1758,  401,   10])}
